In [1]:
import numpy as np
import tensorflow as tf

In [2]:
model_name = 'intercropping'

### Load the data

In [3]:
NP_DATA_PATH = '/mnt/azrael/spatialdata/projects/eo_data/iclr-2020-challange/mlhub/prepared_data/original_data.npz'

In [4]:
data = np.load(NP_DATA_PATH)
features = data['features']
field_id = data['field_id']
crop_id = data['crop_id']

features.shape, field_id.shape, crop_id.shape

((67557, 13, 14), (67557,), (67557,))

### Normalizing the data

In [5]:
mean_f = features.mean(axis=(0,1))
std_f = features.std(axis=(0,1))

mean_f, std_f

(array([0.04495004, 0.05522608, 0.08520251, 0.08335783, 0.13549377,
        0.25449966, 0.29252816, 0.29083016, 0.31650684, 0.31871104,
        0.26029542, 0.17459822, 0.5274778 , 3.82845028]),
 array([ 0.02151336,  0.023685  ,  0.02388389,  0.03328747,  0.03058358,
         0.0464853 ,  0.05491788,  0.05862865,  0.05730477,  0.05223269,
         0.05640826,  0.05882478,  0.18024394, 17.91834208]))

In [6]:
features = (features - mean_f)/std_f

## Spatial sampling

In [7]:
agg_methods = {
    'min': np.mean,
    'max': np.max,
    'mean': np.mean,
    'std': np.std
}

def sample_field(features, N=10, n_samples=1, methods=['mean']):
    agg_features_stack = []
    indices = np.arange(len(features))
    for _ in range(n_samples):
        sampled_i = np.random.choice(indices, size=N, replace=True)

        sampled_features = features[sampled_i]
        
        aggreagations = [agg_methods[method](sampled_features, axis=0) for method in methods]
        agg_features = np.concatenate(aggreagations, axis=-1)
        
        agg_features_stack.append(agg_features)
    
    return np.stack(agg_features_stack)

In [8]:
from tqdm.auto import tqdm

def sample_dataset(features, labels, field_ids, n=10, methods=['mean']):
    fields, counts = np.unique(field_ids, return_counts=True)
    
    features_sampled = []
    labels_sampled = []
    field_id_sampled = []
    weights_sampled = []
    for fid, c in zip(tqdm(fields), counts):
        n_samples = max(c//n, 3)

        field_mask = field_ids == fid
        field_features = features[field_mask]
        field_label = labels[field_mask][0]
        
        sampled_features = sample_field(field_features, N=n, n_samples=n_samples, methods=methods)
        sampled_labels = np.full(n_samples, field_label)
        sampled_fid = np.full(n_samples, fid)
        sampled_weights = np.full(n_samples, 1.0/n_samples)
        
        features_sampled.append(sampled_features)
        labels_sampled.append(sampled_labels)
        field_id_sampled.append(sampled_fid)
        weights_sampled.append(sampled_weights)
    
    features = np.concatenate(features_sampled, axis=0)
    labels = np.concatenate(labels_sampled, axis=0)
    field_ids = np.concatenate(field_id_sampled, axis=0)
    weights = np.concatenate(weights_sampled, axis=0)
    
    return features, labels, field_ids, weights

In [20]:
features_s, crop_id_s, field_id_s, weights_s = sample_dataset(features, crop_id, field_id, methods=['mean'])

In [21]:
features_s.shape

(14755, 13, 14)

### Extract training data and split train, val

In [22]:
mask = crop_id_s != 0
features_valid = features_s[mask]
labels_valid = crop_id_s[mask] - 1
field_id_valid = field_id_s[mask]
weights_valid = weights_s[mask]

features_test = features_s[~mask]
field_id_test = field_id_s[~mask]

features_valid.shape, features_test.shape

((10363, 13, 14), (4392, 13, 14))

In [23]:
# # Compute field split and save
# val_ratio = 0.1
# random_state = np.random.RandomState(seed=42)

# fields = np.unique(field_id_valid)
# random_state.shuffle(fields)

# val_i = int(val_ratio * len(fields))
# fields_train = fields[val_i:]
# fields_val = fields[:val_i]
# fields_test = np.unique(field_id_test)

# # Save split
# np.save('/mnt/azrael/spatialdata/projects/eo_data/iclr-2020-challange/mlhub/split/train.npy', fields_train)
# np.save('/mnt/azrael/spatialdata/projects/eo_data/iclr-2020-challange/mlhub/split/val.npy', fields_val)
# np.save('/mnt/azrael/spatialdata/projects/eo_data/iclr-2020-challange/mlhub/split/test.npy', fields_test)
# fields_train.shape, fields_val.shape, fields_test.shape

In [24]:
# Load training field IDs (split)
fields_train = np.load('/mnt/azrael/spatialdata/projects/eo_data/iclr-2020-challange/mlhub/split/train.npy')

In [25]:
train_mask = np.isin(field_id_valid, fields_train)
print(f'Ratio of pixels in train: {np.mean(train_mask)}')

features_train = features_valid[train_mask]
labels_train = labels_valid[train_mask]
field_id_train = field_id_valid[train_mask]
weights_train = weights_valid[train_mask]

features_val = features_valid[~train_mask]
labels_val = labels_valid[~train_mask]
field_id_val = field_id_valid[~train_mask]
weights_val = weights_valid[~train_mask]

features_train.shape, features_val.shape

Ratio of pixels in train: 0.8999324519926662


((9326, 13, 14), (1037, 13, 14))

## Concat features from intercropping

In [26]:
cropping_dir = '/mnt/azrael/spatialdata/projects/eo_data/iclr-2020-challange/mlhub/prepared_data/mixed_features'

In [43]:
features4 = np.load(f'{cropping_dir}/features4.npy')
features5 = np.load(f'{cropping_dir}/features5.npy')
features7 = np.load(f'{cropping_dir}/features7.npy')

labels4 = np.load(f'{cropping_dir}/labels4.npy') - 1
labels5 = np.load(f'{cropping_dir}/labels5.npy') - 1
labels7 = np.load(f'{cropping_dir}/labels7.npy') - 1

features4.shape, features5.shape, features7.shape

((1000, 13, 14), (1000, 13, 14), (1000, 13, 14))

In [67]:
features_train_ext = np.concatenate([features_train, features4, features5, features7], axis=0)
labels_train_ext = np.concatenate([labels_train, labels4, labels5, labels7], axis=0)
intrecropping_weights = 1.0
weights_train_ext = np.concatenate([weights_train, 
                                    np.ones_like(labels4) * 0.33, 
                                    np.ones_like(labels5) * 0.15,
                                    np.ones_like(labels7) * 0.075])

In [68]:
# Flattened features
features_train_flat = features_train_ext.reshape((features_train_ext.shape[0], -1))
features_val_flat = features_val.reshape((features_val.shape[0], -1))

### Train model

In [69]:
from eoflow.models import TempCNNModel, BiRNN, TransformerEncoder

In [70]:
# TempCNN model

model_config = {
    'learning_rate': 0.1,
    'n_classes': 7,
    'keep_prob': 0.5,
    'nb_conv_stacks': 2
}
model = TempCNNModel(model_config)

In [71]:
# BiRNN model

# model_config = {
#     'learning_rate': 0.1,
#     'n_classes': 7,
#     'rnn_layer': 'lstm',
#     'keep_prob': 0.5
# }
# model = BiRNN(model_config)

In [72]:
# Dense Model

# model = tf.keras.Sequential([
#     tf.keras.layers.Dense(256, activation='relu'),
#     tf.keras.layers.Dropout(rate=0.5),
#     tf.keras.layers.Dense(7, activation='softmax')
# ])

In [73]:
# Class weights
# classes = np.unique(labels_valid, return_counts=True)
# class_probs = classes[1]/np.sum(classes[1])
# class_inv_probs = 1.0 / class_probs
# weights = {i:inv_prob for i, inv_prob in enumerate(class_inv_probs)}
# weights

In [74]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [75]:
# Temporal
model.fit(
    x=features_train_ext, 
    y=labels_train_ext, 
    validation_data=(features_val, labels_val),
    batch_size=256,
    epochs=250,
    sample_weight=weights_train_ext
)

# Non-temporal
# model.fit(
#     x=features_train_flat, 
#     y=labels_train, 
#     validation_data=(features_val_flat, labels_val),
#     batch_size=1024,
#     epochs=10)

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 13, 14)]          0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 13, 16)            1136      
_________________________________________________________________
activation_6 (Activation)    (None, 13, 16)            0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 13, 16)            0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 13, 16)            1296      
_________________________________________________________________
activation_7 (Activation)    (None, 13, 16)            0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 13, 16)            0   

2020-03-10 16:01:53,868 WARNING sample_weight modes were coerced from
  ...
    to  
  ['...']


Train on 12326 samples, validate on 1037 samples
Epoch 1/250
12326/12326 [==============================] - 2s 159us/sample - loss: 0.8554 - accuracy: 0.2605 - val_loss: 1.5453 - val_accuracy: 0.4417
Epoch 2/250
12326/12326 [==============================] - 1s 68us/sample - loss: 0.5346 - accuracy: 0.3234 - val_loss: 1.5784 - val_accuracy: 0.4600
Epoch 3/250
12326/12326 [==============================] - 1s 66us/sample - loss: 0.4864 - accuracy: 0.3535 - val_loss: 1.5528 - val_accuracy: 0.4581
Epoch 4/250
12326/12326 [==============================] - 1s 65us/sample - loss: 0.4637 - accuracy: 0.3717 - val_loss: 1.5231 - val_accuracy: 0.4552
Epoch 5/250
12326/12326 [==============================] - 1s 66us/sample - loss: 0.4491 - accuracy: 0.3900 - val_loss: 1.5063 - val_accuracy: 0.4513
Epoch 6/250
12326/12326 [==============================] - 1s 64us/sample - loss: 0.4376 - accuracy: 0.3962 - val_loss: 1.4795 - val_accuracy: 0.4590
Epoch 7/250
12326/12326 [=========================

Epoch 55/250
12326/12326 [==============================] - 1s 67us/sample - loss: 0.3459 - accuracy: 0.5046 - val_loss: 1.2177 - val_accuracy: 0.5612
Epoch 56/250
12326/12326 [==============================] - 1s 65us/sample - loss: 0.3439 - accuracy: 0.5073 - val_loss: 1.2116 - val_accuracy: 0.5689
Epoch 57/250
12326/12326 [==============================] - 1s 66us/sample - loss: 0.3464 - accuracy: 0.5024 - val_loss: 1.2125 - val_accuracy: 0.5661
Epoch 58/250
12326/12326 [==============================] - 1s 66us/sample - loss: 0.3435 - accuracy: 0.5098 - val_loss: 1.2080 - val_accuracy: 0.5689
Epoch 59/250
12326/12326 [==============================] - 1s 65us/sample - loss: 0.3438 - accuracy: 0.5064 - val_loss: 1.2110 - val_accuracy: 0.5689
Epoch 60/250
12326/12326 [==============================] - 1s 66us/sample - loss: 0.3431 - accuracy: 0.5069 - val_loss: 1.2196 - val_accuracy: 0.5593
Epoch 61/250
12326/12326 [==============================] - 1s 66us/sample - loss: 0.3408 - ac

12326/12326 [==============================] - 1s 64us/sample - loss: 0.3298 - accuracy: 0.5210 - val_loss: 1.2003 - val_accuracy: 0.5747
Epoch 110/250
12326/12326 [==============================] - 1s 66us/sample - loss: 0.3267 - accuracy: 0.5237 - val_loss: 1.1848 - val_accuracy: 0.5892
Epoch 111/250
12326/12326 [==============================] - 1s 68us/sample - loss: 0.3285 - accuracy: 0.5186 - val_loss: 1.1905 - val_accuracy: 0.5873
Epoch 112/250
12326/12326 [==============================] - 1s 67us/sample - loss: 0.3267 - accuracy: 0.5234 - val_loss: 1.1995 - val_accuracy: 0.5844
Epoch 113/250
12326/12326 [==============================] - 1s 66us/sample - loss: 0.3272 - accuracy: 0.5214 - val_loss: 1.1935 - val_accuracy: 0.5844
Epoch 114/250
12326/12326 [==============================] - 1s 66us/sample - loss: 0.3261 - accuracy: 0.5233 - val_loss: 1.1980 - val_accuracy: 0.5863
Epoch 115/250
12326/12326 [==============================] - 1s 64us/sample - loss: 0.3256 - accuracy:

Epoch 163/250
12326/12326 [==============================] - 1s 63us/sample - loss: 0.3172 - accuracy: 0.5295 - val_loss: 1.1734 - val_accuracy: 0.6037
Epoch 164/250
12326/12326 [==============================] - 1s 65us/sample - loss: 0.3189 - accuracy: 0.5291 - val_loss: 1.1776 - val_accuracy: 0.5979
Epoch 165/250
12326/12326 [==============================] - 1s 66us/sample - loss: 0.3203 - accuracy: 0.5287 - val_loss: 1.1696 - val_accuracy: 0.5969
Epoch 166/250
12326/12326 [==============================] - 1s 65us/sample - loss: 0.3201 - accuracy: 0.5235 - val_loss: 1.1880 - val_accuracy: 0.5988
Epoch 167/250
12326/12326 [==============================] - 1s 64us/sample - loss: 0.3168 - accuracy: 0.5296 - val_loss: 1.1826 - val_accuracy: 0.6066
Epoch 168/250
12326/12326 [==============================] - 1s 65us/sample - loss: 0.3170 - accuracy: 0.5307 - val_loss: 1.1628 - val_accuracy: 0.5940
Epoch 169/250
12326/12326 [==============================] - 1s 64us/sample - loss: 0.31

Epoch 217/250
12326/12326 [==============================] - 1s 65us/sample - loss: 0.3123 - accuracy: 0.5350 - val_loss: 1.1669 - val_accuracy: 0.6066
Epoch 218/250
12326/12326 [==============================] - 1s 63us/sample - loss: 0.3113 - accuracy: 0.5393 - val_loss: 1.1650 - val_accuracy: 0.6172
Epoch 219/250
12326/12326 [==============================] - 1s 67us/sample - loss: 0.3127 - accuracy: 0.5324 - val_loss: 1.1715 - val_accuracy: 0.6162
Epoch 220/250
12326/12326 [==============================] - 1s 65us/sample - loss: 0.3089 - accuracy: 0.5353 - val_loss: 1.1718 - val_accuracy: 0.6181
Epoch 221/250
12326/12326 [==============================] - 1s 64us/sample - loss: 0.3104 - accuracy: 0.5351 - val_loss: 1.1667 - val_accuracy: 0.6104
Epoch 222/250
12326/12326 [==============================] - 1s 64us/sample - loss: 0.3097 - accuracy: 0.5361 - val_loss: 1.1583 - val_accuracy: 0.6181
Epoch 223/250
12326/12326 [==============================] - 1s 63us/sample - loss: 0.31

### Random forest

In [57]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss

random_state = 7
rf = RandomForestClassifier(n_estimators=500, random_state=random_state, n_jobs=-1)

rf.fit(features_train_flat, labels_train_ext)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=-1, oob_score=False, random_state=7, verbose=0,
                       warm_start=False)

### Predictions

In [76]:
preds_rf = rf.predict_proba(features_val_flat)
preds_dnn = model.predict(features_val)

In [77]:
preds_rf.shape, preds_dnn.shape

((1037, 7), (1037, 7))

In [78]:
def one_hot(labels, num_classes):
    """ One-hot encodes integer labels for a given number of classes. """
    
    one_hot_labels = np.zeros((labels.size, num_classes))
    one_hot_labels[np.arange(labels.size),labels] = 1
    
    return one_hot_labels

In [79]:
import pandas as pd
def field_predictions(predictions, labels, field_ids):
    """ Groups pixel-wise predictions into field predictions by averaging. """
    
    df = pd.DataFrame(predictions)
    df['CROP_ID'] = labels
    df['FIELD_ID'] = field_ids
    
    df_grouped = df.groupby('FIELD_ID').mean()
    field_preds = df_grouped.drop('CROP_ID', axis=1).values
    field_labels = df_grouped['CROP_ID'].values
    
    return field_preds, field_labels, df_grouped.index

In [80]:
# Create onehot labels for val
one_hot_labels_val = one_hot(labels_val, 7)

### Pixel log loss

In [81]:
rf_loss = log_loss(one_hot_labels_val, preds_rf)
dnn_loss = log_loss(one_hot_labels_val, preds_dnn)

print(f'Random forest: {rf_loss}')
print(f'DNN: {dnn_loss}')

Random forest: 1.230002812363958
DNN: 1.1610276711271248


### Field log loss

In [82]:
preds_rf_field, lbl_rf_field, field_id_rf = field_predictions(preds_rf, labels_val, field_id_val)
preds_dnn_field, lbl_dnn_field, field_id_dnn = field_predictions(preds_dnn, labels_val, field_id_val)

In [83]:
lbl_rf_field_oh = one_hot(lbl_rf_field, 7)
lbl_dnn_field_oh = one_hot(lbl_dnn_field, 7)

In [84]:
rf_loss_field = log_loss(lbl_rf_field_oh, preds_rf_field)
dnn_loss_field = log_loss(lbl_dnn_field_oh, preds_dnn_field)

print(f'Random forest: {rf_loss_field}')
print(f'DNN: {dnn_loss_field}')

Random forest: 1.233688965660122
DNN: 1.1546287469724874


## Prepare predictions for analysis

In [85]:
import pandas as pd

In [86]:
def prepare_predictions(preds, labels, field_ids):
    preds_l = np.argmax(preds, axis=1)

    df = pd.DataFrame(preds, columns=[f'Crop_ID_{i+1}' for i in range(7)])
    df['Field_ID'] = field_ids
    df['prediction'] = preds_l+1
    df['label'] = labels+1
    
    df = df.set_index('Field_ID')
    
    return df

In [87]:
df_rf = prepare_predictions(preds_rf_field, lbl_rf_field, field_id_rf)
df_dnn = prepare_predictions(preds_dnn_field, lbl_dnn_field, field_id_dnn)
df_rf

,Crop_ID_1,Crop_ID_2,Crop_ID_3,Crop_ID_4,Crop_ID_5,Crop_ID_6,Crop_ID_7,prediction,label
Field_ID,,,,,,,,,
1,0.472000,0.318000,0.016000,0.078000,0.044000,0.064000,0.008000,1,1
27,0.481333,0.328667,0.029333,0.048000,0.031333,0.077333,0.004000,1,2
33,0.266000,0.308000,0.162000,0.111333,0.066667,0.036000,0.050000,2,2
44,0.301333,0.560667,0.009333,0.074000,0.016667,0.028667,0.009333,2,1
50,0.226000,0.512000,0.013333,0.115333,0.086000,0.034000,0.013333,2,5
...,...,...,...,...,...,...,...,...,...
4717,0.557333,0.266000,0.016000,0.060667,0.068000,0.026667,0.005333,1,1
4733,0.478200,0.081000,0.026600,0.223600,0.052400,0.105400,0.032800,1,1
4735,0.286000,0.187333,0.034000,0.192000,0.080000,0.214000,0.006667,1,4


In [88]:
np.mean(df_dnn['prediction'] == df_dnn['label'])

0.6219512195121951

In [89]:
df_dnn.to_csv(f'/mnt/azrael/spatialdata/projects/eo_data/iclr-2020-challange/mlhub/val_predictions/{model_name}.csv')

## Prepare results for submission

In [53]:
# Train best model on whole training dataset (without val)
# TempCNN model

model_config = {
    'learning_rate': 0.1,
    'n_classes': 7,
    'keep_prob': 0.5,
    'nb_conv_stacks': 2
}
model_final = TempCNNModel(model_config)
model_final.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [54]:
model_final.fit(
    x=features_valid, 
    y=labels_valid,
    batch_size=256,
    epochs=250,
    sample_weight=weights_valid,
)

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 13, 42)]          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 13, 16)            3376      
_________________________________________________________________
activation_3 (Activation)    (None, 13, 16)            0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 13, 16)            0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 13, 16)            1296      
_________________________________________________________________
activation_4 (Activation)    (None, 13, 16)            0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 13, 16)            0   

2020-03-10 10:48:16,323 WARNING sample_weight modes were coerced from
  ...
    to  
  ['...']


Train on 10363 samples
Epoch 1/250
10363/10363 [==============================] - 2s 165us/sample - loss: 0.7074 - accuracy: 0.3393
Epoch 2/250
10363/10363 [==============================] - 1s 68us/sample - loss: 0.5031 - accuracy: 0.4262
Epoch 3/250
10363/10363 [==============================] - ETA: 0s - loss: 0.4835 - accuracy: 0.45 - 1s 67us/sample - loss: 0.4829 - accuracy: 0.4528
Epoch 4/250
10363/10363 [==============================] - 1s 69us/sample - loss: 0.4697 - accuracy: 0.4740
Epoch 5/250
10363/10363 [==============================] - 1s 68us/sample - loss: 0.4578 - accuracy: 0.4931
Epoch 6/250
10363/10363 [==============================] - 1s 67us/sample - loss: 0.4521 - accuracy: 0.5049
Epoch 7/250
10363/10363 [==============================] - 1s 68us/sample - loss: 0.4494 - accuracy: 0.5146
Epoch 8/250
10363/10363 [==============================] - 1s 67us/sample - loss: 0.4427 - accuracy: 0.5187
Epoch 9/250
10363/10363 [==============================] - 1s 68us/sam

10363/10363 [==============================] - 1s 73us/sample - loss: 0.3516 - accuracy: 0.6056
Epoch 150/250
10363/10363 [==============================] - 1s 70us/sample - loss: 0.3533 - accuracy: 0.5983
Epoch 151/250
10363/10363 [==============================] - 1s 69us/sample - loss: 0.3524 - accuracy: 0.6083
Epoch 152/250
10363/10363 [==============================] - 1s 73us/sample - loss: 0.3506 - accuracy: 0.6051
Epoch 153/250
10363/10363 [==============================] - 1s 68us/sample - loss: 0.3494 - accuracy: 0.6094
Epoch 154/250
10363/10363 [==============================] - 1s 71us/sample - loss: 0.3536 - accuracy: 0.6069
Epoch 155/250
10363/10363 [==============================] - 1s 67us/sample - loss: 0.3486 - accuracy: 0.6079
Epoch 156/250
10363/10363 [==============================] - 1s 65us/sample - loss: 0.3495 - accuracy: 0.6074
Epoch 157/250
10363/10363 [==============================] - 1s 68us/sample - loss: 0.3507 - accuracy: 0.6091
Epoch 158/250
10363/1036

Test on val

In [55]:
preds_val = model_final.predict(features_val)

In [56]:
one_hot_labels_val = one_hot(labels_val, 7)
log_loss(one_hot_labels_val, preds_val)

0.8696593886087294

Predict on test data

In [60]:
preds_final = model_final.predict(features_test)

In [61]:
# Groups pixel-wise predictions into field predictions by averaging.
    
df_pred = pd.DataFrame(preds_final, columns=[f'Crop_ID_{i+1}' for i in range(7)])
df_pred['Field_ID'] = field_id_test
    
df_grouped = df_pred.groupby('Field_ID').mean()
df_grouped

,Crop_ID_1,Crop_ID_2,Crop_ID_3,Crop_ID_4,Crop_ID_5,Crop_ID_6,Crop_ID_7
Field_ID,,,,,,,
3,0.093878,0.741820,0.018600,0.058965,0.010796,0.007072,0.068869
6,0.092141,0.853336,0.000131,0.013566,0.013991,0.025687,0.001148
11,0.087642,0.674603,0.010626,0.132801,0.056422,0.003938,0.033968
13,0.243931,0.547923,0.016267,0.029965,0.114173,0.034280,0.013460
14,0.048479,0.865151,0.042108,0.007048,0.008549,0.000287,0.028379
...,...,...,...,...,...,...,...
4785,0.909692,0.070041,0.006883,0.007005,0.002077,0.002031,0.002272
4788,0.922520,0.023131,0.007554,0.028200,0.004992,0.013580,0.000022
4790,0.790623,0.102938,0.008065,0.026936,0.042148,0.028407,0.000883


In [62]:
df_grouped.to_csv('submissions/temp_cnn_sampling_weighted.csv')

In [63]:
model_final.save_weights('models/temp_cnn_sampling_weighted.hdf5')